In [ ]:
import os
import sys
import pandas as pd
import numpy as np

np.random.seed(777)

In [ ]:
train = pd.read_csv('./data/train.csv')
D_code = pd.read_csv('./data/속성_D_코드.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv("./data/sample_submission.csv")

In [ ]:
# 속성 D 1번, 2번, 3번의 대분류, 코드 매칭 여부 변수 생성을 위한 데이터 처리
d1 = D_code.copy()
d1 = d1.iloc[:,[0,4]]
d1.columns = ['person_prefer_d_1', 'person_prefer_d_1_l']
d2 = D_code.copy()
d2 = d2.iloc[:,[0,4]]
d2.columns = ['person_prefer_d_2', 'person_prefer_d_2_l']
d3 = D_code.copy()
d3 = d3.iloc[:,[0,4]]
d3.columns = ['person_prefer_d_3', 'person_prefer_d_3_l']
con_d = D_code.copy()
con_d = con_d.iloc[:,[0,4]]
con_d.columns = ['contents_attribute_d', 'contents_attribute_d_l']

# 전처리를 위한 함수 생성
def preprocessing_1(data):
    # Bool to category
    for i in range(len(data.columns)):
        if data.dtypes[i] == 'bool':
            data.iloc[:,i] = data.iloc[:,i].apply(lambda x: 1 if x==True else 0)

    # 날짜 변수 생성
    data["contents_open_dt"] = pd.to_datetime(data["contents_open_dt"])
    data["contents_open_wd"] = data.contents_open_dt.dt.weekday
    data["contents_open_hour"] = data.contents_open_dt.dt.hour
    data['contents_weekday'] = data.apply(lambda x: 1 if x['contents_open_wd'] not in [5, 6] else 0, axis=1) # 평일 및 주말
    data['contents_work_time'] = data.apply(lambda x: 1 if x['contents_open_hour'] in [10,11,12,13,14,15,16,17] else 0, axis=1) # 시간대

    # 컨텐츠 번호 빈도수 변수 생성
    contents_rn_cnt=data.groupby(data["contents_rn"]).size()
    contents_rn_cnt.name="contents_rn_cnt"
    data = pd.merge(data, contents_rn_cnt, on="contents_rn")

    # 사용자 번호 빈도수 변수 생성
    person_rn_cnt=data.groupby(data["person_rn"]).size()
    person_rn_cnt.name="person_rn_cnt"
    data = pd.merge(data, person_rn_cnt, on="person_rn")

    # 속성 D merge
    data = pd.merge(data, d1, on="person_prefer_d_1")
    data = pd.merge(data, d2, on="person_prefer_d_2")
    data = pd.merge(data, d3, on="person_prefer_d_3")
    data = pd.merge(data, con_d, on="contents_attribute_d")
    
    return data

# 전처리
train = preprocessing_1(train)
test = preprocessing_1(test)

# 속성 D의 1번, 2번, 3번의 대분류 및 코드 매칭 여부 변수를 만드는 함수
def match(data, col1, col2, col3):
    data[col1] = data.apply(lambda x: 1 if x[col2]==x[col3] else 0, axis=1)
    
# 속성 D의 대분류 매칭 여부 변수 생성
match(train, "d_1_l_match_yn", "person_prefer_d_1_l", "contents_attribute_d_l")
match(train, "d_2_l_match_yn", "person_prefer_d_2_l", "contents_attribute_d_l")
match(train, "d_3_l_match_yn", "person_prefer_d_3_l", "contents_attribute_d_l")
match(test, "d_1_l_match_yn", "person_prefer_d_1_l", "contents_attribute_d_l")
match(test, "d_2_l_match_yn", "person_prefer_d_2_l", "contents_attribute_d_l")
match(test, "d_3_l_match_yn", "person_prefer_d_3_l", "contents_attribute_d_l")

# 속성 D의 코드 매칭 여부 변수 생성
match(train, "d_1_s_match_yn", "person_prefer_d_1", "contents_attribute_d")
match(train, "d_2_s_match_yn", "person_prefer_d_2", "contents_attribute_d")
match(train, "d_3_s_match_yn", "person_prefer_d_3", "contents_attribute_d")
match(test, "d_1_s_match_yn", "person_prefer_d_1", "contents_attribute_d")
match(test, "d_2_s_match_yn", "person_prefer_d_2", "contents_attribute_d")
match(test, "d_3_s_match_yn", "person_prefer_d_3", "contents_attribute_d")

# id에 대하여 데이터 정렬
train = train.sort_values(by='id')
test = test.sort_values(by='id')

# 필요 없는 열 제거
''' 아이디(id), label이 하나인 경우('person_prefer_f', 'person_prefer_g'),
    사용자 번호(person_rn), 컨텐츠 번호(contents_rn), 컨텐츠 열람 일시(contents_open_dt),
    기존에 있었던 속성 D 매칭 여부 변수(d_l_match_yn, d_m_match_yn, d_s_match_yn), 속성 H 중분류 및 코드 매칭 여부(h_m_match_yn, h_s_match_yn),
    merge 하면서 생긴 속성 D 대분류 코드(person_prefer_d_1_l, person_prefer_d_2_l, person_prefer_d_3_l, contents_attribute_d_l)'''

train.drop(['id','person_prefer_f','person_prefer_g','person_rn','contents_rn','contents_open_dt',
            'd_l_match_yn','d_m_match_yn','d_s_match_yn', "h_m_match_yn", "h_s_match_yn",
            'person_prefer_d_1_l', 'person_prefer_d_2_l', 'person_prefer_d_3_l', 'contents_attribute_d_l'], axis=1, inplace=True)

test.drop(['id','person_prefer_f','person_prefer_g','person_rn','contents_rn','contents_open_dt',
            'd_l_match_yn','d_m_match_yn','d_s_match_yn', "h_m_match_yn", "h_s_match_yn",
            'person_prefer_d_1_l', 'person_prefer_d_2_l', 'person_prefer_d_3_l', 'contents_attribute_d_l'], axis=1, inplace=True)

# 인덱스 재정렬
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

train.to_csv('./data/train_data.csv')
train.to_csv('./data/test_data.csv')

- 아이디, label이 하나인 경우, 사용자 번호, 컨텐츠 번호, 컨텐츠 열람 일시 변수의 경우 모델 생성에 필요하지 않은 변수
- 기존에 있었던 속성 D 매칭 여부 변수는 속성 D의 1번, 2번, 3번의 대분류 및 코드 매칭 여부 변수를 따로 만들어주었으므로 제거해줌
- 속성 H 중분류 및 코드 매칭 여부 변수는 속성 H 대분류 매칭 여부 변수와 관련성이 높으며, 앞서 시각화에서 봤듯이 속성 H 코드 매칭 여부 변수가 target 변수 분류에 큰 영향을 주지 않을 것이라고 판단해 속성 H 대분류 매칭 여부 변수만 남겨줌